In [1]:
import utils
from utils import *

trainPath=r"C:\Users\sivateja_k\Desktop\NLP_Project\dataset\Reviews.csv"
review,sentiment,filtered_data=data_preprocess(trainPath)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sivateja_k\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


resulting filtered data length after dropping rating 3 should be 525814
Filtered data length is 525814


C:\Users\sivateja_k\Desktop\NLP_Project\utils.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['sentiment']=filtered_data['Score'].apply(assign_score)
C:\Users\sivateja_k\Anaconda3\envs\nlpproject\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [2]:
#Text Summarization starts here
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
#Now lets try to inspect some reviews and their summary
for i in range(3):
    print("========================================")
    print(filtered_data.Summary[i])
    print()
    print(review[i])
str1=" "
final_string=[]
s=""
for sentence in filtered_data["Summary"].values:
    filtered_sentence=[]
    sentence=cleanHTMLTags(sentence)
    sentence=contractions.fix(sentence)
    sentence=cleanPunctuations(sentence)
#     words=[snow.stem(word) for word in sentence.split()]
#     str1=" ".join(words)
    final_string.append(sentence.lower())

C:\Users\sivateja_k\Anaconda3\envs\nlpproject\lib\site-packages\tensorflow\python\framework\dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sivateja_k\Anaconda3\envs\nlpproject\lib\site-packages\tensorflow\python\framework\dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sivateja_k\Anaconda3\envs\nlpproject\lib\site-packages\tensorflow\python\framework\dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sivateja_k\Anaconda3\envs\nlpproje

Good Quality Dog Food

i bought sever vital can dog food product found good qualiti the product look like stew process meat smell better my labrador finicki appreci product better
Not as Advertised

product arriv label jumbo salt peanut peanut actual small size unsalt not sure error vendor intend repres product jumbo
"Delight" says it all

this confect around centuri it light pillowi citrus gelatin nut - case filbert and cut tini squar liber coat powder sugar and tini mouth heaven not chewi flavor i high recommend yummi treat if familiar stori c s lewi the lion the witch the wardrob - treat seduc edmund sell brother sister witch


In [3]:
summary=final_string
final_string[0:3]

['good quality dog food', 'not as advertised', ' delight  says it all']

In [4]:
embeddings_index = {}
#loading conceptNumber batch embedding (CN)
with open(r'C:\Users\sivateja_k\Desktop\NLP_Project\libraries\numberbatch-en.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embeddings_index[word] = np.asarray(values[1:], dtype='float32')
print(list(embeddings_index.keys())[10000:10010])
#Now lets collect all the words in review +summary so we will get the total vocabulary of our data 
#then we should see if our vocab is completly present in the embeddings_index or not
vocab={}
for sentence in summary:
    for word in sentence.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for sentence in review:
    for word in sentence.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

['charles_digby_harrod', 'xanthium', 'sandpaper', 'residentially', 'be_alike', 'microdispersion', 'euxine', 'several_weeks', 'roncesvalles', 'cargo_container']


In [5]:
missing_words = 0
threshold = 25

for word, count in vocab.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(vocab),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 5118
Percent of words that are missing from vocabulary: 3.16%


In [6]:
vocab_to_int = {} 

value = 0
for word, count in vocab.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1


for token in ["<UNK>","<PAD>","<EOS>","<GO>"]:
    vocab_to_int[token] = value
    value+=1

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

print("Total number of unique words:", len(vocab))
print("Number of words we will use:", len(vocab_to_int))

Total number of unique words: 161865
Number of words we will use: 41561


In [7]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits


def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state[0],
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32)) 
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)

    return training_logits, inference_logits

def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths
        
def text_to_seq(text):
    '''Prepare the text for the model'''
    text=cleanHTMLTags(text)
    text=contractions.fix(text)
    text=cleanPunctuations(text)
#     text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]







     
    

In [8]:
embedding_dim = len(embeddings_index[list(embeddings_index.keys())[1]])
word_embedding_matrix = np.zeros((len(vocab_to_int), embedding_dim), dtype=np.float32)
print(word_embedding_matrix.shape)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        word_embedding_matrix[i] =  np.array(np.random.uniform(-1.0, 1.0, embedding_dim))

# both lengths should be same
print(len(word_embedding_matrix))
print(len(vocab_to_int))

(41561, 300)
41561
41561


In [9]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0
int_summaries, word_count, unk_count = convert_to_ints(summary, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(review, word_count, unk_count, eos=True)
print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)

Total number of words in headlines: 18013072
Total number of UNKs in headlines: 266559


In [10]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
              counts
count  364159.000000
mean        4.106291
std         2.635843
min         0.000000
25%         2.000000
50%         3.000000
75%         5.000000
max        48.000000

Texts:
              counts
count  364159.000000
mean       46.358563
std        44.339332
min         2.000000
25%        21.000000
50%        33.000000
75%        56.000000
max      1716.000000


In [11]:
sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 100 # use 1
unk_summary_limit = 100 # use 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

282316
282316


In [12]:
# Set the Hyperparameters
import tensorflow as tf
epochs = 2 # use 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Graph is built.


In [13]:
# Train the Model
import time
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = r"C:\Users\sivateja_k\Desktop\NLP_Project\libraries\best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries, sorted_texts, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/2 Batch   20/4411 - Loss:  4.046, Seconds: 58.68
Epoch   1/2 Batch   40/4411 - Loss:  2.706, Seconds: 56.21
Epoch   1/2 Batch   60/4411 - Loss:  2.522, Seconds: 55.75
Epoch   1/2 Batch   80/4411 - Loss:  2.531, Seconds: 57.93
Epoch   1/2 Batch  100/4411 - Loss:  2.542, Seconds: 37.46
Epoch   1/2 Batch  120/4411 - Loss:  2.581, Seconds: 49.70
Epoch   1/2 Batch  140/4411 - Loss:  2.474, Seconds: 31.58
Epoch   1/2 Batch  160/4411 - Loss:  2.424, Seconds: 65.61
Epoch   1/2 Batch  180/4411 - Loss:  2.251, Seconds: 44.83
Epoch   1/2 Batch  200/4411 - Loss:  2.368, Seconds: 56.34
Epoch   1/2 Batch  220/4411 - Loss:  2.300, Seconds: 34.78
Epoch   1/2 Batch  240/4411 - Loss:  2.306, Seconds: 36.39
Epoch   1/2 Batch  260/4411 - Loss:  2.216, Seconds: 50.90
Epoch   1/2 Batch  280/4411 - Loss:  2.234, Seconds: 50.05
Epoch   1/2 Batch  300/4411 - Loss:  2.150, Seconds: 37.19
Epoch   1/2 Batch  320/4411 - Loss:  2.485, Seconds: 52.11
Epoch   1/2 Batch  340/4411 - Loss:  2.310, Seconds: 62.

Epoch   1/2 Batch 2800/4411 - Loss:  1.941, Seconds: 75.86
Epoch   1/2 Batch 2820/4411 - Loss:  2.020, Seconds: 57.32
Epoch   1/2 Batch 2840/4411 - Loss:  2.026, Seconds: 64.59
Epoch   1/2 Batch 2860/4411 - Loss:  1.923, Seconds: 67.28
Epoch   1/2 Batch 2880/4411 - Loss:  2.006, Seconds: 50.37
Epoch   1/2 Batch 2900/4411 - Loss:  1.996, Seconds: 73.94
Epoch   1/2 Batch 2920/4411 - Loss:  2.003, Seconds: 61.76
Average loss for this update: 2.0
New Record!
Epoch   1/2 Batch 2940/4411 - Loss:  2.036, Seconds: 63.05
Epoch   1/2 Batch 2960/4411 - Loss:  2.126, Seconds: 66.23
Epoch   1/2 Batch 2980/4411 - Loss:  1.940, Seconds: 62.94
Epoch   1/2 Batch 3000/4411 - Loss:  1.974, Seconds: 70.79
Epoch   1/2 Batch 3020/4411 - Loss:  2.077, Seconds: 73.90
Epoch   1/2 Batch 3040/4411 - Loss:  1.998, Seconds: 91.91
Epoch   1/2 Batch 3060/4411 - Loss:  2.029, Seconds: 75.85
Epoch   1/2 Batch 3080/4411 - Loss:  2.066, Seconds: 67.80
Epoch   1/2 Batch 3100/4411 - Loss:  2.023, Seconds: 73.03
Epoch   1/

Epoch   2/2 Batch 1160/4411 - Loss:  1.730, Seconds: 46.17
Epoch   2/2 Batch 1180/4411 - Loss:  1.762, Seconds: 60.53
Epoch   2/2 Batch 1200/4411 - Loss:  1.694, Seconds: 55.93
Epoch   2/2 Batch 1220/4411 - Loss:  1.849, Seconds: 50.75
Epoch   2/2 Batch 1240/4411 - Loss:  1.795, Seconds: 53.48
Epoch   2/2 Batch 1260/4411 - Loss:  1.766, Seconds: 43.87
Epoch   2/2 Batch 1280/4411 - Loss:  1.819, Seconds: 59.41
Epoch   2/2 Batch 1300/4411 - Loss:  1.646, Seconds: 48.31
Epoch   2/2 Batch 1320/4411 - Loss:  1.757, Seconds: 65.78
Epoch   2/2 Batch 1340/4411 - Loss:  1.840, Seconds: 55.27
Epoch   2/2 Batch 1360/4411 - Loss:  1.792, Seconds: 44.68
Epoch   2/2 Batch 1380/4411 - Loss:  1.717, Seconds: 50.70
Epoch   2/2 Batch 1400/4411 - Loss:  1.793, Seconds: 53.96
Epoch   2/2 Batch 1420/4411 - Loss:  1.707, Seconds: 59.37
Epoch   2/2 Batch 1440/4411 - Loss:  1.746, Seconds: 52.45
Epoch   2/2 Batch 1460/4411 - Loss:  1.797, Seconds: 71.24
Average loss for this update: 1.741
New Record!
Epoch   

Epoch   2/2 Batch 3900/4411 - Loss:  1.952, Seconds: 83.22
Epoch   2/2 Batch 3920/4411 - Loss:  2.001, Seconds: 71.35
Epoch   2/2 Batch 3940/4411 - Loss:  1.932, Seconds: 79.75
Epoch   2/2 Batch 3960/4411 - Loss:  2.062, Seconds: 59.69
Epoch   2/2 Batch 3980/4411 - Loss:  1.935, Seconds: 74.52
Epoch   2/2 Batch 4000/4411 - Loss:  1.980, Seconds: 78.22
Epoch   2/2 Batch 4020/4411 - Loss:  2.029, Seconds: 78.95
Epoch   2/2 Batch 4040/4411 - Loss:  2.010, Seconds: 89.95
Epoch   2/2 Batch 4060/4411 - Loss:  1.916, Seconds: 91.71
Epoch   2/2 Batch 4080/4411 - Loss:  1.908, Seconds: 86.21
Epoch   2/2 Batch 4100/4411 - Loss:  1.938, Seconds: 85.50
Epoch   2/2 Batch 4120/4411 - Loss:  2.009, Seconds: 77.00
Epoch   2/2 Batch 4140/4411 - Loss:  2.055, Seconds: 96.05
Epoch   2/2 Batch 4160/4411 - Loss:  2.014, Seconds: 87.31
Epoch   2/2 Batch 4180/4411 - Loss:  2.075, Seconds: 92.72
Epoch   2/2 Batch 4200/4411 - Loss:  2.080, Seconds: 91.90
Epoch   2/2 Batch 4220/4411 - Loss:  2.042, Seconds: 103

In [14]:
input_sentence = "love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets"
text = text_to_seq(input_sentence)
# random = np.random.randint(0,len(review))
# input_sentence = review[random]
# text = text_to_seq(review[random])

checkpoint = r"C:\Users\sivateja_k\Desktop\NLP_Project\libraries\best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
   # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

   # Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                     summary_length: [np.random.randint(5,8)], 
                                     text_length: [len(text)]*batch_size,
                                     keep_prob: 1.0})[0] 

#Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('\nOriginal Text:', input_sentence)

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from C:\Users\sivateja_k\Desktop\NLP_Project\libraries\best_model.ckpt

Original Text: love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually trilled get entire case time go anywhere need water microwave spoon know quaker flavor packets

Text
  Word Ids:    [13065, 32926, 32687, 28632, 37721, 8221, 1991, 18112, 5410, 296, 26278, 12334, 1632, 5410, 296, 37721, 25302, 40586, 36131, 13134, 41557, 22051, 11847, 18284, 31956, 3945, 233, 4209, 10485, 1879, 31885, 16032, 27808, 25948, 38993]
  Input Words: love individual oatmeal cups found years ago sam quit selling sound big lots quit selling found target expensive buy individually <UNK> get entire case time go anywhere need water microwave spoon know quaker flavor packets

Summary
  Word Ids:       [40679, 9853]
  Response Words: great product
